# Parsing votes
> Downloading & parsing votes Aafter downloading xlsx files behind the links on `https://www.bundestag.de/parlament/plenum/abstimmung/liste`.

In [1]:
%load_ext autoreload
%autoreload 2

## Setup

In [2]:
from fastcore.all import *
from bundestag import html_parsing as hp

## Collecting URIs for `.xlsx`/`.xls` documents from `.htm` files

`.xlsx` / `.xls` will be referred as "sheet" files.

In [3]:
html_path = Path('../website_data')
sheet_path = Path('../sheets')

In [6]:
html_file_paths = hp.get_file_paths(html_path, pattern=hp.RE_HTM)
html_file_paths[:3]

(#3) [Path('../website_data/Deutscher Bundestag - Namenslisten der Abstimmungen (PDF und XLSX-Listen)-1.htm'),Path('../website_data/Deutscher Bundestag - Namenslisten der Abstimmungen (PDF und XLSX-Listen)-10.htm'),Path('../website_data/Deutscher Bundestag - Namenslisten der Abstimmungen (PDF und XLSX-Listen)-11.htm')]

In [8]:
hp.test_file_paths(html_file_paths, html_path)

In [9]:
%%time
sheet_uris = hp.collect_sheet_uris(html_file_paths)
list(sheet_uris.items())[:3], list(sheet_uris.items())[-3:]

HTM(L): 100%|███████████████████████████████████████████████████████████████████████████| 24/24 [00:09<00:00,  2.64it/s]

CPU times: user 8.02 s, sys: 69.9 ms, total: 8.09 s
Wall time: 9.08 s


([('10.09.2020: Abstrakte Normenkontrolle - Düngeverordnung (Beschlussempfehlung)',
   'https://www.bundestag.de/resource/blob/791698/852a05058b1d4614945d0f38b4ecb224/20200910_2_xls-data.xlsx'),
  ('03.07.2020: Bundeswahlgesetz (Geschäftsordnungsantrag)',
   'https://www.bundestag.de/resource/blob/704496/5d6e83f4c2204ac3a514b7970fff9a48/20200703_2_xls-data.xlsx'),
  ('03.07.2020: Kohleausstiegsgesetz (Änderungsantrag 19/20730)',
   'https://www.bundestag.de/resource/blob/704410/abaa35a8f21b571771856b759602d9df/20200703_1_xls-data.xlsx')],
 [('25.10.2012: 17/10059 und 17/11093, Abkommen zwischen Deutschland und der Schweiz',
   'https://www.bundestag.de/resource/blob/192676/2ffed073c6ac4a2bd600b83523e8d5a1/20121025_2_xls-data.xls'),
  ('25.10.2012: 17/10773 und 17/11174, Änderungen im Bereich der geringfügigen Beschäftigung',
   'https://www.bundestag.de/resource/blob/192674/db858b70d2821545c109b8c5ae752722/20121025_xls-data.xls'),
  ('18.10.2012: Gesetzentwurf 17/9852 und 17/11053 (8. 

In [10]:
hp.test_sheet_uris(sheet_uris)

## Downloading sheet files

In [11]:
%%time
uri = sheet_uris['10.09.2020: Abstrakte Normenkontrolle - Düngeverordnung (Beschlussempfehlung)']
hp.download_sheet(uri, sheet_path=sheet_path, verbose=True)

Writing to ../sheets/20200910_2_xls-data.xlsx
CPU times: user 24.2 ms, sys: 536 µs, total: 24.7 ms
Wall time: 253 ms


In [12]:
%%time
file_title_maps = hp.download_multiple_sheets(sheet_uris, sheet_path=sheet_path, nmax=3)

File:   1%|▋                                                                           | 4/464 [00:00<00:01, 453.76it/s]

CPU times: user 7.09 ms, sys: 972 µs, total: 8.07 ms
Wall time: 12.1 ms


In [14]:
hp.test_file_title_maps(file_title_maps, sheet_uris)

## Loading sheets into DataFrames

Collecting the `xlsx` and `xls` file names

In [15]:
sheet_files = hp.get_file_paths(sheet_path, pattern=hp.RE_FNAME)
sheet_files

(#464) [Path('../sheets/20121018_xls-data.xls'),Path('../sheets/20121025_2_xls-data.xls'),Path('../sheets/20121025_3_xls-data.xls'),Path('../sheets/20121025_4_xls-data.xls'),Path('../sheets/20121025_5_xls-data.xls'),Path('../sheets/20121025_xls-data.xls'),Path('../sheets/20121109_betreuungsgeld_xls-data.xls'),Path('../sheets/20121109_praxisgebuehr_xls-data.xls'),Path('../sheets/20121121_1_xls-data.xls'),Path('../sheets/20121121_2_xls-data.xls')...]

In [16]:
assert len(sheet_files) > 0

Reading files into dataframes

In [18]:
%%time
sheet_file = sheet_files[0]
df = hp.get_sheet_df(sheet_file, file_title_maps=file_title_maps)
df.head().T

CPU times: user 39.1 ms, sys: 1.13 ms, total: 40.3 ms
Wall time: 54.6 ms


,0,1,2,3,4
Wahlperiode,17,17,17,17,17
Sitzungnr,198,198,198,198,198
Abstimmnr,1,1,1,1,1
Fraktion/Gruppe,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU
Name,Aigner,Altmaier,Aumer,Bär,Bareiß
Vorname,Ilse,Peter,Peter,Dorothee,Thomas
Titel,NaN,NaN,NaN,NaN,NaN
ja,1,0,1,0,1
nein,0,0,0,0,0
Enthaltung,0,0,0,0,0


In [19]:
hp.test_get_sheet_df(df)

Squishing vote columns

In [21]:
df_squished = hp.get_squished_dataframe(df)
df_squished.head().T

/mnt/c/PetProjects/bundestag/nbs/bundestag/html_parsing.py:138: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  tmp = (tmp[tmp == 1].stack()


,0,1,2,3,4
Wahlperiode,17,17,17,17,17
Sitzungnr,198,198,198,198,198
Abstimmnr,1,1,1,1,1
Fraktion/Gruppe,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU
Name,Aigner,Altmaier,Aumer,Bär,Bareiß
Vorname,Ilse,Peter,Peter,Dorothee,Thomas
Titel,NaN,NaN,NaN,NaN,NaN
Bezeichnung,Ilse Aigner,Peter Altmaier,Peter Aumer,Dorothee Bär,Thomas Bareiß
sheet_name,T_Export,T_Export,T_Export,T_Export,T_Export
date,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00


In [ ]:
hp.test_squished_df(df_squished, df)

Setting some dtypes

In [22]:
df_squished = hp.set_sheet_dtypes(df_squished)

Loading multiple sheets into dataframes

In [24]:
%%time
df = hp.get_multiple_sheets_df(sheet_files, file_title_maps=file_title_maps)
df.head().T

Sheets:   0%|                                                                                   | 0/464 [00:00<?, ?it/s]/mnt/c/PetProjects/bundestag/nbs/bundestag/html_parsing.py:138: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  tmp = (tmp[tmp == 1].stack()
Sheets:  12%|████████▊                                                                 | 55/464 [00:03<00:21, 18.81it/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Sheets:  14%|██████████▎                                                               | 65/464 [00:03<00:22, 17.82it/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Sheets:  21%|███████████████▍                                                          | 97/464 [00:05<00:19, 18.43it/s]

...

Sheets:  76%|███████████████████████████████████████████████████████▌                 | 353/464 [00:19<00:06, 16.54it/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Sheets: 100%|█████████████████████████████████████████████████████████████████████████| 464/464 [00:40<00:00, 11.42it/s]


CPU times: user 29.1 s, sys: 432 ms, total: 29.6 s
Wall time: 40.9 s


,0,1,2,3,4
Wahlperiode,17,17,17,17,17
Sitzungnr,198,198,198,198,198
Abstimmnr,1,1,1,1,1
Fraktion/Gruppe,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU
Name,Aigner,Altmaier,Aumer,Bär,Bareiß
Vorname,Ilse,Peter,Peter,Dorothee,Thomas
Titel,nan,nan,nan,nan,nan
Bezeichnung,Ilse Aigner,Peter Altmaier,Peter Aumer,Dorothee Bär,Thomas Bareiß
sheet_name,T_Export,T_Export,T_Export,T_Export,T_Export
date,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00


Doing all the above

In [25]:
%%time
df = hp.get_multiple_sheets(html_path, sheet_path, nmax=3)
df.head().T

Sheets:   0%|                                                                                   | 0/464 [00:00<?, ?it/s]/mnt/c/PetProjects/bundestag/nbs/bundestag/html_parsing.py:138: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  tmp = (tmp[tmp == 1].stack()
Sheets:  12%|████████▌                                                                 | 54/464 [00:03<00:22, 17.96it/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Sheets:  14%|██████████▌                                                               | 66/464 [00:03<00:23, 16.99it/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Sheets:  21%|███████████████▋                                                          | 98/464 [00:05<00:20, 17.48it/s]

...

Sheets:  76%|███████████████████████████████████████████████████████▋                 | 354/464 [00:20<00:06, 16.01it/s]

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


Sheets: 100%|█████████████████████████████████████████████████████████████████████████| 464/464 [00:37<00:00, 12.41it/s]


CPU times: user 39.8 s, sys: 530 ms, total: 40.3 s
Wall time: 46.9 s


,0,1,2,3,4
Wahlperiode,17,17,17,17,17
Sitzungnr,198,198,198,198,198
Abstimmnr,1,1,1,1,1
Fraktion/Gruppe,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU,CDU/CSU
Name,Aigner,Altmaier,Aumer,Bär,Bareiß
Vorname,Ilse,Peter,Peter,Dorothee,Thomas
Titel,nan,nan,nan,nan,nan
Bezeichnung,Ilse Aigner,Peter Altmaier,Peter Aumer,Dorothee Bär,Thomas Bareiß
sheet_name,T_Export,T_Export,T_Export,T_Export,T_Export
date,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00,2012-10-18 00:00:00


Writing to disk

In [ ]:
%%time
# df.to_parquet('../roll_call_votes.parquet')